### Imports

In [ ]:
!pip install fair-esm
!pip install torch

In [ ]:
import pandas as pd
import pickle
import esm
import torch
import numpy as np

In [ ]:
dir = "/content/drive/MyDrive/Duke/Freshman Year/Sem 2/BME 590/Shrey Goel/Individual Project 2A/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Combine all fine-tuned ProtGPT2 generation results

In [ ]:
# open each batch (1k sequences) of generated sequences
dfs = []
for i in range(1, 11):
  path = dir + "finetuned_protgpt2_gen_results/df" + str(i) + ".pkl"
  with open(path, "rb") as f:
        dfs.append(pd.read_pickle(f))

In [ ]:
data = pd.concat(dfs, ignore_index=True)
data

,Sequence,Perplexity
0,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,10.683612
1,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107
2,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,9.774858
3,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAITVTPSGEVSH...,1.927557
4,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAITVTPSGEVSH...,32.980544
...,...,...
9995,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIFATPEGEVSH...,8.729079
9996,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107
9997,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,5.464747
9998,<|endoftext|>\nMDEIDELSDLPTPRFIWGFAIAVTPSGEVSH...,1.011107


In [ ]:
# strip processing tags to only have sequence
import re
data['Sequence'] = data['Sequence'].str.replace('\n', '')
data['Sequence'] = data['Sequence'].str.replace('<|endoftext|>', '')
data['Sequence'] = data['Sequence'].str.lstrip("||")

In [ ]:
data

,Sequence,Perplexity
0,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,10.683612
1,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107
2,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.774858
3,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,1.927557
4,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,32.980544
...,...,...
9995,MDEIDELSDLPTPRFIWGFAIFATPEGEVSHDEFEYLTHTRAPRFT...,8.729079
9996,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107
9997,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,5.464747
9998,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107


In [ ]:
data.to_csv(dir + "ppls_10k_gen_seqs.csv", index=False)

### Load SVC classifier and data

In [ ]:
"""
It is necessary to filter for anti-crispr sequences using the SVC because we want our downstream ESM-2 model fine-tuned on the MLM task to understand
anti-crispr sequence relationships and unmask residues that contribute to anti-crispr stability.
"""

In [ ]:
# load saved SVC classifier from Individual Project 1
with open(dir + "/anti_crispr_classification_model/anti_crispr_classifier.pkl", "rb") as f:
  anti_crispr_classifier = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
data = pd.read_csv(dir + "ppls_10k_gen_seqs.csv")

In [ ]:
data = data.drop(columns=['Unnamed: 0'])

In [ ]:
data

,Sequence,Perplexity
0,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,10.683612
1,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107
2,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.774858
3,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,1.927557
4,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,32.980544
...,...,...
9995,MDEIDELSDLPTPRFIWGFAIFATPEGEVSHDEFEYLTHTRAPRFT...,8.729079
9996,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107
9997,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,5.464747
9998,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107


In [ ]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results
model.cuda() # push model to GPU

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [ ]:
# generate ESM embeddings for generated anti-criprs
# ESM embeddings are needed as these are the inputs to the SVC classifier
sequence_data = pd.DataFrame(data['Sequence'])
avg_embeddings = []

for index, row in sequence_data.iterrows():
    sequence = row["Sequence"]

    batch_labels, batch_strs, batch_tokens = batch_converter([("", sequence)])
    batch_tokens = batch_tokens.cuda()
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6].cpu()

    avg_embedding = token_representations[0, 1 : batch_lens[0] - 1].mean(0)
    avg_embeddings.append(avg_embedding.numpy())

gen_embeddings = pd.DataFrame(avg_embeddings)
gen_embeddings

,0,1,2,3,4,5,6,7,8,9,...,310,311,312,313,314,315,316,317,318,319
0,0.020003,0.023015,0.117911,0.056656,0.275330,0.072467,-0.026653,-0.091976,-0.063911,-0.175051,...,0.057898,0.087591,-0.166147,0.190436,0.184715,-0.201323,0.045333,0.155860,-0.038875,-0.078417
1,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193
2,0.061048,0.013061,0.070764,-0.044064,0.284801,0.091927,0.043827,-0.073432,-0.061483,-0.165392,...,0.088418,0.046299,-0.079256,0.148867,0.190142,-0.161899,-0.025283,0.083232,0.084679,-0.141900
3,0.049017,-0.016382,0.020665,-0.062750,0.306721,0.131980,0.089089,-0.072200,-0.103067,-0.149651,...,0.134008,0.069034,-0.117115,0.143493,0.153234,-0.215196,-0.018252,0.090487,0.066269,-0.170939
4,0.058438,0.144647,0.156240,-0.171163,0.285640,0.127019,0.103459,-0.112572,-0.165082,-0.216944,...,0.264804,0.129107,-0.135641,0.118916,0.137686,-0.082792,0.102473,0.123905,0.129777,-0.196628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.080742,0.023363,0.053867,-0.009773,0.253669,0.093287,-0.027720,-0.042545,-0.104995,-0.173100,...,0.061835,0.026784,-0.087012,0.127683,0.174025,-0.177072,0.029103,0.061301,0.065738,-0.092112
9996,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193
9997,0.123437,0.040257,0.055015,0.001512,0.308516,0.154398,0.047405,-0.055844,-0.052292,-0.181210,...,0.088118,0.103465,-0.131082,0.139730,0.179803,-0.232549,-0.040965,0.056909,0.122391,-0.136396
9998,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193


In [ ]:
gen_embeddings

,0,1,2,3,4,5,6,7,8,9,...,310,311,312,313,314,315,316,317,318,319
0,0.020003,0.023015,0.117911,0.056656,0.275330,0.072467,-0.026653,-0.091976,-0.063911,-0.175051,...,0.057898,0.087591,-0.166147,0.190436,0.184715,-0.201323,0.045333,0.155860,-0.038875,-0.078417
1,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193
2,0.061048,0.013061,0.070764,-0.044064,0.284801,0.091927,0.043827,-0.073432,-0.061483,-0.165392,...,0.088418,0.046299,-0.079256,0.148867,0.190142,-0.161899,-0.025283,0.083232,0.084679,-0.141900
3,0.049017,-0.016382,0.020665,-0.062750,0.306721,0.131980,0.089089,-0.072200,-0.103067,-0.149651,...,0.134008,0.069034,-0.117115,0.143493,0.153234,-0.215196,-0.018252,0.090487,0.066269,-0.170939
4,0.058438,0.144647,0.156240,-0.171163,0.285640,0.127019,0.103459,-0.112572,-0.165082,-0.216944,...,0.264804,0.129107,-0.135641,0.118916,0.137686,-0.082792,0.102473,0.123905,0.129777,-0.196628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.080742,0.023363,0.053867,-0.009773,0.253669,0.093287,-0.027720,-0.042545,-0.104995,-0.173100,...,0.061835,0.026784,-0.087012,0.127683,0.174025,-0.177072,0.029103,0.061301,0.065738,-0.092112
9996,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193
9997,0.123437,0.040257,0.055015,0.001512,0.308516,0.154398,0.047405,-0.055844,-0.052292,-0.181210,...,0.088118,0.103465,-0.131082,0.139730,0.179803,-0.232549,-0.040965,0.056909,0.122391,-0.136396
9998,0.062309,-0.014091,0.064261,-0.038880,0.286109,0.109276,0.059534,-0.077127,-0.074450,-0.170408,...,0.133362,0.059386,-0.082048,0.141722,0.156706,-0.175021,-0.058296,0.057345,0.097790,-0.144193


In [ ]:
data.insert(2, "avg_embedding", avg_embeddings)
data

,Sequence,Perplexity,avg_embedding
0,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,10.683612,"[0.020002997, 0.023014687, 0.11791095, 0.05665..."
1,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03..."
2,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.774858,"[0.061047968, 0.013060541, 0.07076411, -0.0440..."
3,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,1.927557,"[0.049016897, -0.01638242, 0.020664962, -0.062..."
4,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,32.980544,"[0.058437947, 0.14464717, 0.15624048, -0.17116..."
...,...,...,...
9995,MDEIDELSDLPTPRFIWGFAIFATPEGEVSHDEFEYLTHTRAPRFT...,8.729079,"[0.08074203, 0.02336324, 0.053867254, -0.00977..."
9996,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03..."
9997,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,5.464747,"[0.12343735, 0.040257215, 0.05501532, 0.001511..."
9998,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03..."


### Classify sequences

In [ ]:
# use SVC to classify each generated sequence
for idx, embedding in data['avg_embedding'].items():
  prediction = anti_crispr_classifier.predict(embedding.reshape(1, -1))
  data.at[idx, 'classifer_prediction'] = prediction

In [ ]:
# filter for only anti-crispr-classified sequences
svc_classified_anti_crisprs = data[data['classifer_prediction'] != 0.0].reset_index()
svc_classified_anti_crisprs

,index,Sequence,Perplexity,avg_embedding,classifer_prediction
0,159,MTTFTSIVTTNPDFRTFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,24.429898,"[0.050432272, 0.0662099, 0.14025089, -0.017260...",1.0
1,585,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,3.940383,"[0.091618754, -0.027570218, 0.15275551, -0.086...",1.0
2,619,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.767750,"[0.10370322, 0.027244542, 0.16705583, -0.10965...",1.0
3,926,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,3.302805,"[0.08464557, -0.030284416, 0.15467276, -0.0997...",1.0
4,1684,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,10.619778,"[0.10430293, 0.025759643, 0.16828096, -0.11159...",1.0
5,1969,MTTFTSIVTTNPDFGGFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,1.312910,"[-0.018298369, 0.016527716, 0.2024873, -0.1390...",1.0
6,2057,MTTFTSIVTTNPDFGGFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,14.974984,"[0.12530063, 0.16374947, 0.1399832, -0.0784045...",1.0
7,2449,MTTFTSIVTTNPDFGGFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,14.572710,"[0.10029177, 0.15128724, 0.22198136, -0.116690...",1.0
8,3433,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,7.753764,"[0.08141662, 0.019493131, 0.1419369, -0.111959...",1.0
9,3933,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,6.822530,"[0.08616534, 0.07485319, 0.13415046, -0.127030...",1.0


In [ ]:
# filter for only crispr-classified sequences
svc_classified_crisprs = data[data['classifer_prediction'] == 0.0].reset_index()
svc_classified_crisprs

,index,Sequence,Perplexity,avg_embedding,classifer_prediction
0,0,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,10.683612,"[0.020002997, 0.023014687, 0.11791095, 0.05665...",0.0
1,1,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03...",0.0
2,2,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.774858,"[0.061047968, 0.013060541, 0.07076411, -0.0440...",0.0
3,3,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,1.927557,"[0.049016897, -0.01638242, 0.020664962, -0.062...",0.0
4,4,MDEIDELSDLPTPRFIWGFAITVTPSGEVSHDEFEYLTHTRAPRFT...,32.980544,"[0.058437947, 0.14464717, 0.15624048, -0.17116...",0.0
...,...,...,...,...,...
9971,9995,MDEIDELSDLPTPRFIWGFAIFATPEGEVSHDEFEYLTHTRAPRFT...,8.729079,"[0.08074203, 0.02336324, 0.053867254, -0.00977...",0.0
9972,9996,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03...",0.0
9973,9997,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,5.464747,"[0.12343735, 0.040257215, 0.05501532, 0.001511...",0.0
9974,9998,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,1.011107,"[0.062308628, -0.014090952, 0.064261176, -0.03...",0.0


In [ ]:
# save classification results as csv
svc_classified_anti_crisprs.to_csv(dir + "svc_classified_anti_crisprs.csv")
svc_classified_crisprs.to_csv(dir + "svc_classified_crisprs.csv")

### Identify top 10 anti-CRISPR sequences based on perplexity scores

In [ ]:
# rank anti-crispr sequences based on lowest pPL
svc_classified_anti_crisprs_top10 = svc_classified_anti_crisprs.nsmallest(10, 'Perplexity').reset_index()
svc_classified_anti_crisprs_top10 = svc_classified_anti_crisprs_top10.drop(columns=['index', 'level_0'])
svc_classified_anti_crisprs_top10

,Sequence,Perplexity,avg_embedding,classifer_prediction
0,MTTFTSIVTTNPDFGGFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,1.312910,"[-0.018298369, 0.016527716, 0.2024873, -0.1390...",1.0
1,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,2.693028,"[0.08720274, -0.033054944, 0.16769509, -0.0820...",1.0
2,MDEIDELSDLPTPRFIWGFAIAVTPDGKGGFEYISHDEYEAMTDEE...,3.134373,"[0.07362139, 0.039984465, 0.06761675, -0.06659...",1.0
3,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,3.302805,"[0.08464557, -0.030284416, 0.15467276, -0.0997...",1.0
4,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,3.458252,"[-0.012192436, 0.08646252, 0.0765099, 0.038242...",1.0
5,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,3.940383,"[0.091618754, -0.027570218, 0.15275551, -0.086...",1.0
6,MTTFTSIVTTNPDFGGFEFYVEAGQQFDDSAYEEAYGVSVPTAVVE...,4.411345,"[0.018114936, 0.0818875, 0.1465818, -0.1652928...",1.0
7,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,6.822530,"[0.08616534, 0.07485319, 0.13415046, -0.127030...",1.0
8,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,7.753764,"[0.08141662, 0.019493131, 0.1419369, -0.111959...",1.0
9,MDEIDELSDLPTPRFIWGFAIAVTPSGEVSHDEFEYLTHTRSPRFT...,9.310184,"[0.10423464, 0.04585983, 0.15291446, -0.101685...",1.0


In [ ]:
# save top 10 sequences as csv
svc_classified_anti_crisprs_top10.to_csv(dir + "ppls_top10_gen_seqs.csv")